# Econ 8210 Quant Macro, Homework 1
## Part 3 - Dynamic Programming
Haosi Shen, Fall 2024

In [5]:
import numpy as np
import math
import time
import torch
from scipy.optimize import fsolve
from scipy.interpolate import interp1d


# import numba
# from numba import jit
# @jit(nopython=True)
np.random.seed(42) 

Consider the following model. There is a representative household with preferences over private consumption $c_t$, government consumption $g_t$, and labor $l_t$:

\begin{equation*}
\mathbb{E}_0 \sum_{t=0}^{\infty} 0.97^t \Big( \log{c_t}+0.2\log{g_t} -\frac{l_t^2}{2} \Big)
\end{equation*}

The household consumes, saves, works, and pays labor taxes, with a budget constraint:

\begin{equation*}
c_t + i_t = (1-\tau_t)w_t l_t + r_t k_t
\end{equation*}

where the tax rate follows a 3-state Markov chain that takes values in:
\begin{equation*}
\tau_t \in \{0.20, 0.25, 0.30\}
\end{equation*}


There is a production function:
\begin{equation*}
c_t + i_t + g_t = e^{z_t} k_{t}^{0.33} l_{t}^{0.67}
\end{equation*}

with a law of motion for capital with investment adjustment costs:
\begin{equation*}
k_{t+1} = 0.9 k_t + \Big(1-0.05 \big(\frac{i_t}{i_{t-1}}-1\big)^2 \Big) i_t
\end{equation*}

and a technology level $z_t$ that follows a 5-state Markov chain that takes values in:
\begin{equation*}
z_t \in \{−0.0673, −0.0336, 0, 0.0336, 0.0673\}
\end{equation*}

Finally, there is a government that uses taxes to pay for government consumption with a
balanced budget period by period:
\begin{equation*}
g_t = \tau_t w_t l_t
\end{equation*}

### Social Planner's Problem

The social planner solves

\begin{equation*}
\max_{\{c_t, l_t, i_t\}} \mathbb{E}_0 \left[ \sum_{t=0}^{\infty} \beta^t \Big( \log{c_t}+0.2\log{g_t} -\frac{l_t^2}{2} \Big) \right]
\end{equation*}
subject to:
\begin{align*}
c_t + i_t + \tau_t w_t l_t &= e^{z_t} k_{t}^{\alpha} l_{t}^{1-\alpha} \\
k_{t+1} &= (1-\delta) k_t + \Big(1-0.05 \big(\frac{i_t}{i_{t-1}}-1\big)^2 \Big) i_t
\end{align*}
where $\beta = 0.97,\; \alpha=0.33,\;\delta=0.1 $.

**Recursive Formulation**
> *State Variables*: $\; k_t,\; i_{t-1},\; \tau_t,\; z_t$ \
> *Control Variables*: $\; c_t,\; i_t,\; l_t$

**Bellman Equation**

\begin{equation*}
V(k,i_{-1},\tau,z) = \max_{c,l,i}\; \left\{ U(c,l) + \beta \sum_{\tau', z'} \pi(\tau', z' | \tau, z) V(k',i,\tau',z') \right\}
\end{equation*}

where:
\begin{align*}
k' &= (1-\delta) k + \Big(1-0.05 \big(\frac{i}{i_{-1}}-1\big)^2 \Big) i \\
U(c,l) &= \log{c} + 0.2 \log{\big(\tau(1-\alpha)e^z k^\alpha l^{1-\alpha} \big)}-\frac{l^2}{2} \\
c &= \big(1-\tau(1-\alpha)\big) e^z k^\alpha l^{1-\alpha} -i
\end{align*}

**Revised Bellman Equation**

\begin{align*}
V(k,i_{-1},\tau,z) = \max_{l,i}\; & \Bigg\{ \log{\bigg(\big(1-\tau(1-\alpha)\big) e^z k^\alpha l^{1-\alpha} -i\bigg)} + 0.2 \log{\big(\tau(1-\alpha)e^z k^\alpha l^{1-\alpha} \big)}-\frac{l^2}{2} \\
& + \beta \sum_{\tau', z'} \pi(\tau', z' | \tau, z)\; V\bigg((1-\delta) k + \Big(1-0.05 \big(\frac{i}{i_{-1}}-1\big)^2 \Big) i,i,\tau',z'\bigg) \Bigg\}
\end{align*}


### Deterministic Steady State

Given $\tau_{ss}=0.25,\; z_{ss}=0$.

To solve for $c_{ss}, l_{ss}, i_{ss}, k_{ss}$, we solve the following system of equations

* **Implementability Condition** (i.e. optimal labor under distortionary taxes)
$$ l_{ss} = \frac{w_{ss}(1-0.25)}{c_{ss}} $$
* Production Function
$$ y_{ss} = k_{ss}^{\alpha} l_{ss}^{1-\alpha} $$    
* Resource Constraint
$$ c_{ss}+ \delta k_{ss} + 0.25 w_{ss}l_{ss} = k_{ss}^{\alpha} l_{ss}^{1-\alpha} $$
* Steady State Investment
$$ i_{ss} = \delta k_{ss} $$
* Steady State Wage
$$ w_{ss}=(1-\alpha)\Big(\frac{k_{ss}}{l_{ss}}\Big)^{\alpha} $$



In [2]:
# ===================== Model Parameters =====================
beta = 0.97  # time discount factor
alpha = 0.33  # capital share
delta = 0.1  # capital depreciation rate
theta_c = 0.2  # weight on govt consumption in util

# ===================== Steady-State Parameters =====================
tau_ss = 0.25
z_ss = 0.0

def steady_state_equations(vars):
    c_ss, i_ss, k_ss, l_ss, w_ss = vars

    # Equation 1: Optimal labor condition, with tax distortion
    eq1 = l_ss - w_ss * (1 - tau_ss) / c_ss

    # Equation 2: Production function
    eq2 = c_ss + i_ss + tau_ss * w_ss * l_ss - np.exp(z_ss) * k_ss**alpha * l_ss**(1 - alpha)

    # Equation 3: Capital evolution
    eq3 = k_ss - ((1 - delta) * k_ss + i_ss)

    # Equation 4: Investment in ss
    eq4 = i_ss - delta * k_ss

    # Equation 5: Wage rate
    eq5 = w_ss - (1 - alpha) * np.exp(z_ss) * k_ss**alpha * l_ss**(-alpha)

    return [eq1, eq2, eq3, eq4, eq5]


initial_guess = [1, 0.1, 1, 0.5, 0.5]   # [c_ss, i_ss, k_ss, l_ss, w_ss]

# Solve the system of equations
c_ss, i_ss, k_ss, l_ss, w_ss = fsolve(steady_state_equations, initial_guess)

# Compute the rest of equilibrium objects
r_ss = alpha * np.exp(z_ss) * k_ss**(alpha - 1) * l_ss**(1 - alpha)
y_ss = np.exp(z_ss) * k_ss**alpha * l_ss**(1 - alpha)
g_ss = tau_ss * w_ss * l_ss


print(f"Steady-State Consumption: {c_ss}")
print(f"Steady-State Investment: {i_ss}")
print(f"Steady-State Capital: {k_ss}")
print(f"Steady-State Labor: {l_ss}")
print(f"Steady-State Wage: {w_ss}")
print(f"Steady-State Rental Rate: {r_ss}")
print(f"Steady-State Output: {y_ss}")
print(f"Steady-State Government Spending: {g_ss}")

Steady-State Consumption: 0.7983567053829371
Steady-State Investment: 0.25906523478350146
Steady-State Capital: 2.590652347835015
Steady-State Labor: 0.8941318844019129
Steady-State Wage: 0.9517815805452646
Steady-State Rental Rate: 0.16179640888625949
Steady-State Output: 1.2701765047044309
Steady-State Government Spending: 0.21275456453799213


### Value Function Iteration with Fixed Grid


In [3]:
# ===================== Model Setup =====================

# Tax rates (tau): 3-state Markov chain
vTax = np.array([0.2, 0.25, 0.3])
mtransTax = np.array([[0.9, 0.1, 0.0],
                      [0.05, 0.9, 0.05],
                      [0.0, 0.1, 0.9]])

# Technology levels (z): 5-state Markov chain
vTechnology = np.array([-0.0673, -0.0336, 0, 0.0336, 0.0673])
mtransTechnology = np.array([[0.9727, 0.0273, 0, 0, 0],
                    [0.0041, 0.9806, 0.0153, 0, 0],
                    [0, 0.0082, 0.9836, 0.0082, 0],
                    [0, 0, 0.0153, 0.9806, 0.0041],
                    [0, 0, 0, 0.0273, 0.9727]])

# Period utility
def utility(c, g, l):
    if c > 0 and g > 0:
        return np.log(c) + 0.2 * np.log(g) - 0.5 * (l ** 2)
    else:
        return -np.inf
    # return np.log(c) + 0.2 * np.log(g) - 0.5 * (l ** 2)

# Production function
def production(k, l, z):
    return np.exp(z) * (k ** alpha) * (l ** (1 - alpha))


Fix a grid of 250 points of capital, centered around $k_{ss}$ with a coverage of $\pm 30\%$ of $k_{ss}$ and equally spaced and a grid of 50 points on lagged investment, centered around $i_{ss}$ with a coverage of $\pm 50\%$ of $i_{ss}$ and equally spaced.

In [4]:
# Define Capital Grid
num_k_grid = 250
k_grid = np.linspace(0.7 * k_ss, 1.3 * k_ss, num_k_grid) # ±30% k_ss

# Define Lagged Investment Grid
num_i_grid = 50
i_grid = np.linspace(0.5 * i_ss, 1.5 * i_ss, num_i_grid)  # ±50% i_ss

* Iterate on the value function of the household using linear interpolation until the change in the sup norm between two iterations is less than $10^{−6}$. 
* Compute the policy function. 
* Describe the responses of the economy to a technology shock and a tax shock.

In [ ]:
# Initialize value function
V_init = (np.log(c_ss) + theta_c * np.log(g_ss) - 0.5 * (l_ss ** 2)) / (1 - beta)
V = np.full((num_k_grid, num_i_grid, len(vTax), len(vTechnology)), V_init)

# Helper function to calculate next-period capital
def next_capital(k, i, i_lag):
    return 0.9 * k + (1 - 0.05 * ((i / i_lag) - 1) ** 2) * i

# Bellman iteration
tol = 1e-6  # convergence tolerance
max_iter = 1000  # max iterations

for iter in range(max_iter):
    V_new = np.copy(V)
    diff = 0

    # Loop over all state variables
    for k_idx, k in enumerate(k_grid):
        for i_idx, i_lag in enumerate(i_grid):
            for tau_idx, tau in enumerate(vTax):
                for z_idx, z in enumerate(vTechnology):
                    
                    max_val = -np.inf
                    best_c, best_l, best_i = None, None, None
                    l = l_ss   # initialize labor

                    # Loop over control variables: investment
                    for i in i_grid:
                        # Compute the wage
                        w = (1 - alpha) * np.exp(z) * (k ** alpha) * (l ** (-alpha))

                        # Compute consumption
                        c = (1 - tau * (1 - alpha)) * np.exp(z) * (k ** alpha) * (l ** (1 - alpha)) - i

                        # Ensure feasibility
                        if c > 0:

                            # Optimal labor under distortionary income tax
                            l = w * (1 - tau) / c

                            # Compute consumption
                            c = (1 - tau * (1 - alpha)) * np.exp(z) * (k ** alpha) * (l ** (1 - alpha)) - i
                        
                        if l > 0 and c > 0:

                            # Compute government spending
                            g = tau * w * l

                            # Compute utility
                            utility = np.log(c) + theta_c * np.log(g) - 0.5 * (l ** 2)

                            # Compute next-period capital
                            k_next = next_capital(k, i, i_lag)

                            # Interpolate value function for next period
                            v_next = 0
                            k_next_idx = np.searchsorted(k_grid, k_next) - 1
                            k_next_idx = max(0, min(k_next_idx, num_k_grid - 2))

                            for tau_next_idx, tau_next in enumerate(vTax):
                                for z_next_idx, z_next in enumerate(vTechnology):
                                    prob_tau = mtransTax[tau_idx, tau_next_idx]
                                    prob_z = mtransTechnology[z_idx, z_next_idx]
                                    # Linear interpolation
                                    V_interp = V_new[k_next_idx, i_idx, tau_next_idx, z_next_idx]
                                    v_next += prob_tau * prob_z * V_interp

                            # Calculate total value, normalizing period utility
                            total_val = (1 - beta) * utility + beta * v_next

                            # Update value function if higher value found
                            if total_val > max_val:
                                max_val = total_val
                                best_c, best_l, best_i = c, l, i

                    # Update value function
                    V_new[k_idx, i_idx, tau_idx, z_idx] = max_val
                    diff = max(diff, abs(V_new[k_idx, i_idx, tau_idx, z_idx] - V[k_idx, i_idx, tau_idx, z_idx]))

    # Check for convergence
    if diff < tol:
        print(f'Converged in {iter + 1} iterations')
        break

    # Update the value function
    V = V_new

print('Value Function Iteration complete.')

**Policy Functions**

Extract the optimal policy functions for investment, labor, and consumption from the converged value function.

In [11]:
optimal_policy_c = np.zeros_like(V)
optimal_policy_i = np.zeros_like(V)
optimal_policy_l = np.zeros_like(V)

# Extract optimal policies
for i_k, k in enumerate(k_grid):
    for i_i, i_lag in enumerate(i_grid):
        for i_tau, tau in enumerate(vTax):
            for i_z, z in enumerate(vTechnology):
                if V[i_k, i_i, i_tau, i_z] != 0:
                    optimal_policy_c[i_k, i_i, i_tau, i_z] = best_c
                    optimal_policy_i[i_k, i_i, i_tau, i_z] = best_i
                    optimal_policy_l[i_k, i_i, i_tau, i_z] = best_l

**Impulse Response Functions**

Describe the responses of the economy to a technology shock and a tax shock. 

**VFI, SUPER LOW BUT RUNS.**

In [ ]:
'''
# Initialize value function
V = np.zeros((num_k_grid, num_i_grid, len(vTax), len(vTechnology)))
V_new = np.zeros_like(V)

# Step 3: Iterate on Value Function (Optimized)
max_iter = 1000
tol = 1e-6
sup_norm_diff = tol + 1

# Precompute possible labor values
l_vals = np.linspace(0.1, 2.0, 50)  # Reduced discretization
c_vals = np.linspace(0.1, 2.0, 50)  # Reduced discretization

for iteration in range(max_iter):
    if sup_norm_diff < tol:
        break
    sup_norm_diff = 0

    # Precompute V_interp for each tau and z state
    V_interp = {i_tau: {i_z: interp1d(k_grid, V[:, :, i_tau, i_z], kind='linear', axis=0, fill_value="extrapolate")
                        for i_z in range(len(vTechnology))}
                for i_tau in range(len(vTax))}

    for i_k, k in enumerate(k_grid):
        for i_i, i_lag in enumerate(i_grid):
            for i_tau, tau in enumerate(vTax):
                for i_z, z in enumerate(vTechnology):
                    possible_values = []

                    # Precompute wages and government spending for each labor value
                    for l in l_vals:
                        w = (1 - alpha) * np.exp(z) * (k ** alpha) * (l ** (- alpha))
                        g = tau * w * l

                        for c in c_vals:
                            i_next = production(k, l, z) - c - g

                            if i_next <= 0 or c <= 0:
                                continue

                            # Compute expected continuation value
                            expected_value = 0.0
                            for j_tau, tau_next in enumerate(vTax):
                                for j_z, z_next in enumerate(vTechnology):
                                    prob_tau = mtransTax[i_tau, j_tau]
                                    prob_z = mtransTechnology[i_z, j_z]
                                    V_next = V_interp[j_tau][j_z](k)  # Interpolating on capital grid
                                    expected_value += prob_tau * prob_z * V_next

                            # Ensure expected_value is scalar
                            if np.isscalar(expected_value):
                                # Total utility (ensure scalar)
                                u = utility(c, g, l)
                                total_value = u + beta * expected_value

                                # Append scalar values to possible_values
                                possible_values.append((total_value, float(c), float(i_next), float(l)))


                    if possible_values:
                        best_value, best_c, best_i, best_l = max(possible_values, key=lambda x: x[0])
                        V_new[i_k, i_i, i_tau, i_z] = best_value

                    # Update sup-norm difference
                    sup_norm_diff = max(sup_norm_diff, abs(V_new[i_k, i_i, i_tau, i_z] - V[i_k, i_i, i_tau, i_z]))

    # Update value function
    V = V_new.copy()

    if iteration % 10 == 0:
        print(f"Iteration {iteration}, sup-norm diff = {sup_norm_diff}")

print("Converged.")

'''

### Value Function Iteration with Endogenous Grid

###  Comparison of Grids

### Policy Function Iteration